In [1]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect
from pathlib import Path
import pandas as pd

In [2]:
engine = create_engine("sqlite:///Resources/Obesity_db.sqlite")

Base = automap_base()
Base.prepare(autoload_with=engine)
Base.classes.keys()


[]

In [8]:
conn = engine.connect()
Obesity = pd.read_sql("SELECT * FROM Obesity", conn)

In [4]:
Obesity.head()

,State,Prevalence,95% CI
0,Alabama,39.9,"(37.9, 41.8)"
1,Alaska,33.5,"(31.5, 35.5)"
2,Arizona,31.3,"(30.0, 32.6)"
3,Arkansas,38.7,"(36.6, 40.8)"
4,California,27.6,"(26.1, 29.1)"


In [5]:
Fastfood = pd.read_sql("SELECT * FROM Fastfood", conn)
Fastfood.head()

,State,All fast food restaurants,Full-service restaurants,Subway,Starbucks,McDonalds,Dunkin Donut,Burger King,Taco Bell,Dominos,Wendys,Dairy Queen
0,Alabama,81.7,57.6,8.2,1.8,5.0,0.9,3.5,2.8,2.5,2.0,1.3
1,Alaska,61.9,78.7,7.2,6.6,4.2,0.3,1.1,2.1,1.4,1.2,0.7
2,Arizona,67.9,59.2,5.6,7.3,3.9,1.3,2.5,2.5,1.6,1.4,1.5
3,Arkansas,69.9,67.6,8.4,2.0,5.7,0.3,2.7,3.5,2.2,2.1,1.0
4,California,82.3,80.4,5.7,7.4,3.2,0.3,1.4,2.1,1.4,0.7,0.2


In [7]:
inspector = inspect(engine)
inspector.get_table_names()

['fastfood', 'obesity']

In [11]:
columns_ff = inspector.get_columns('fastfood')
for c in columns:
    print(c['name'], c["type"])

State TEXT
All fast food restaurants TEXT
Full-service restaurants  TEXT
Subway TEXT
Starbucks TEXT
McDonalds TEXT
Dunkin Donut TEXT
Burger King TEXT
Taco Bell TEXT
Dominos TEXT
Wendys TEXT
Dairy Queen TEXT


In [12]:
columns_ob = inspector.get_columns('obesity')
for c in columns:
    print(c['name'], c["type"])

State TEXT
All fast food restaurants TEXT
Full-service restaurants  TEXT
Subway TEXT
Starbucks TEXT
McDonalds TEXT
Dunkin Donut TEXT
Burger King TEXT
Taco Bell TEXT
Dominos TEXT
Wendys TEXT
Dairy Queen TEXT


In [4]:
session = Session(engine)

In [48]:
joined_table = pd.read_sql('Select fastfood.State, fastfood."All fast food restaurants", fastfood.Subway, fastfood.Starbucks, fastfood.McDonalds, fastfood."Dunkin Donut", fastfood."Burger King", fastfood."Taco Bell", fastfood.Dominos, fastfood.Wendys, fastfood."Dairy Queen", obesity.Prevalence, obesity."95% CI" from Fastfood INNER JOIN Obesity ON (Fastfood.State = Obesity.State)', conn)

In [49]:
joined_table.head()

,State,All fast food restaurants,Subway,Starbucks,McDonalds,Dunkin Donut,Burger King,Taco Bell,Dominos,Wendys,Dairy Queen,Prevalence,95% CI
0,Alabama,81.7,8.2,1.8,5.0,0.9,3.5,2.8,2.5,2.0,1.3,39.9,"(37.9, 41.8)"
1,Alabama,81.7,8.2,1.8,5.0,0.9,3.5,2.8,2.5,2.0,1.3,39.9,"(37.9, 41.8)"
2,Alaska,61.9,7.2,6.6,4.2,0.3,1.1,2.1,1.4,1.2,0.7,33.5,"(31.5, 35.5)"
3,Alaska,61.9,7.2,6.6,4.2,0.3,1.1,2.1,1.4,1.2,0.7,33.5,"(31.5, 35.5)"
4,Arizona,67.9,5.6,7.3,3.9,1.3,2.5,2.5,1.6,1.4,1.5,31.3,"(30.0, 32.6)"


In [51]:
json_table = joined_table.to_json()